# Setup

In [1]:
from common_code.augmentation_functions import *
from common_code.get_dataset import *
from common_code.models import *
from common_code.training_loop import *
from common_code.utils import *
import tensorflow as tf




# if training_config["ENABLE_DETERMINISM"]:
#   print(f"Enabling determinism. Seed: {training_config.SEED}.")
#   tf.keras.utils.set_random_seed(training_config.SEED)
#   tf.config.experimental.enable_op_determinism()
# else:
#   print("No determinism enabled.")

In [2]:
# from google.colab import auth

# auth.authenticate_user()
reset_tpu(training_config)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Running on GPU.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse
# !mkdir gcs
# !gcsfuse --implicit-dirs tomasmuzas-phd-datasets gcs

In [3]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: muzas-tomas (tomasmuzas-phd). Use `wandb login --relogin` to force relogin


True

# Training

In [2]:
tf.get_logger().setLevel('ERROR')

# with tf.device("/GPU:0"):
#     for z_min, z_max in [(0.4, 0.5), (0.5, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0), (1.0, 1.1), (1.1, 1.2)]:
#         print("Training for ", z_min)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=8096)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

z_min = 0.4
z_max = 1.2

training_config = {
"ENABLE_DETERMINISM": False,
"SEED": 17,
"AUTO": tf.data.AUTOTUNE,
"TRAIN_BATCH_SIZE": 16,
"TEST_BATCH_SIZE": 16,
"NUMBER_OF_EPOCHS": 500,
"EARLY_STOPPING_TOLERANCE": 10,
"LEARNING_RATE": 1e-4,
"IMAGE_SIZE": 128,
"SHUFFLE_BUFFER": 300000,
"TPU": False,
"AUGMENTATIONS_ROTATE": False,
"AUGMENTATIONS_FLIP_HORIZONTALLY": False,
"AUGMENTATIONS_FLIP_VERTICALLY": False,
"AUGMENTATIONS_ZOOM": False,
"AUGMENTATIONS_RANDOM_NOISE": False,
"AUGMENTATIONS_CENTER_NOISE": False,
"AUGMENTATIONS_OUTSIDE_NOISE": False,
"AUGMENTATIONS_PERLIN_CENTER_NOISE": False,
"USE_ADABELIEF_OPTIMIZER": False,
"GALAXY_DATASET": "SDSS",
"DATASET_PATH": f"../Step 2 - Image dataset/TFDataset/{z_min}-{z_max}",
"Z_MIN": z_min,
"Z_MAX": z_max,
# "TRAIN_DATASET_SIZE": 3072,
# "TEST_DATASET_SIZE": 330,
"STRETCH": False,
"STEPS_PER_EXECUTION": 1,

}
training_config["WANDB_PROJECT_NAME"] = f"Baseline (no augmentations) OBJID logging"
training_config["EXPERIMENT_DESCRIPTION"] = f"baseline_no_augmentations_{z_min}_{z_max}"
training_config["LOCAL_GCP_PATH_BASE"] = f"../../../../Step 3 - Training/"
training_config["REMOTE_GCP_PATH_BASE"] = f"../../../../Step 3 - Training/"

models = [
    {'name': 'cavanagh', 'func': Cavanagh, 'starting_fold': 1},
    {'name': 'dieleman', 'func': Dieleman, 'starting_fold': 1},
    {'name': 'resnet50', 'func': create_ResNet50, 'starting_fold': 1},
    {'name': 'newsimple', 'func': SimpleModel, 'starting_fold': 1}
]

perform_training(models, training_config)

1 Physical GPUs, 1 Logical GPUs
cavanagh FOLD 1
Loading model from ../../../../Step 3 - Training//models/128x128/initial_models/cavanagh
Loss improved. Saving model.


Loss improved. Saving model.


Loss improved. Saving model.


Loss improved. Saving model.


Early stopping


cavanagh FOLD 2
Loading model from ../../../../Step 3 - Training//models/128x128/initial_models/cavanagh
Loss improved. Saving model.


Loss improved. Saving model.


Loss improved. Saving model.


Loss improved. Saving model.
